This is quite the complex data splitting procedure.
The data is split into holdout data, training, validation, and testing.
The training and validation data only contains single-cells that have ground truth labels at the terminal time point.
While testing and holdout data contains cells that do and no not have ground truth labels at the terminal time point.

In [1]:
import itertools
import pathlib

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [2]:
# read in the data
sc_file_path = pathlib.Path("../results/cleaned_sc_profile.parquet").resolve(
    strict=True
)
sc_endpoint_file_path = pathlib.Path(
    "../results/cleaned_endpoint_sc_profile.parquet"
).resolve(strict=True)

train_test_wells_file_path = pathlib.Path(
    "../../5.bulk_timelapse_model/data_splits/train_test_wells.parquet"
).resolve(strict=True)

sc_profile = pd.read_parquet(sc_file_path)
sc_endpoint_profile = pd.read_parquet(sc_endpoint_file_path)
train_test_wells = pd.read_parquet(train_test_wells_file_path)
print(f"sc_profile shape: {sc_profile.shape}")
print(f"sc_endpoint_profile shape: {sc_endpoint_profile.shape}")

sc_profile shape: (182804, 2376)
sc_endpoint_profile shape: (11340, 368)


In [3]:
sc_endpoint_profile.head()

,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,Cells_Texture_Correlation_AnnexinV_3_01_256,Cells_Texture_Correlation_AnnexinV_3_02_256,Cells_Texture_Correlation_AnnexinV_3_03_256,Cells_Texture_Correlation_DNA_3_02_256,Cells_Texture_DifferenceVariance_AnnexinV_3_02_256,Cells_Texture_InverseDifferenceMoment_AnnexinV_3_03_256,Cells_Texture_SumAverage_AnnexinV_3_00_256,Cells_Texture_SumAverage_DNA_3_01_256,Metadata_Well_FOV,Metadata_sc_unique_track_id
0,1,C-09,69,Staurosporine,39.06,positive,1,0002,13.0,23,...,-0.105321,0.258335,0.245637,-0.370941,0.063992,0.382514,-0.135229,2.375118,C-09_0002,C-09_0002_129
1,1,C-09,69,Staurosporine,39.06,positive,1,0002,13.0,25,...,-0.650287,-0.535347,-0.911682,-0.218577,0.378465,0.509536,-0.265312,3.657820,C-09_0002,C-09_0002_39
2,1,C-09,69,Staurosporine,39.06,positive,1,0002,13.0,36,...,-0.153755,0.307227,-0.233445,-0.147660,-0.076404,0.218955,-0.069362,2.324912,C-09_0002,C-09_0002_66
3,1,C-09,69,Staurosporine,39.06,positive,1,0002,13.0,38,...,0.270074,0.500432,0.445759,-0.162800,-0.105987,0.304090,-0.025800,2.870185,C-09_0002,C-09_0002_65
4,1,C-09,69,Staurosporine,39.06,positive,1,0002,13.0,42,...,-0.312933,0.471374,-0.057580,-1.426125,-0.291901,0.369278,-0.142464,0.237744,C-09_0002,C-09_0002_75


In [4]:
sc_profile.head()

,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO,Metadata_sc_unique_track_id,Metadata_Well_FOV,Metadata_sc_unique_track_id_count
0,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,101,...,0.313944,1.126927,-0.143103,0.241127,-0.293259,-0.283715,1.434163,C-02_0001_17,C-02_0001,13
1,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,111,...,0.102750,0.845704,0.083930,-1.990931,-0.030848,-1.033722,-0.942127,C-02_0001_18,C-02_0001,13
2,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,11,...,0.810937,0.300940,-0.228780,1.782329,0.153739,-0.763335,0.725093,C-02_0001_5,C-02_0001,1
3,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,128,...,-0.711263,0.067196,-0.149771,1.405650,0.063245,2.162110,3.187469,C-02_0001_19,C-02_0001,13
4,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,132,...,-0.251130,-1.851114,0.669517,-0.439855,1.576201,0.747753,0.895601,C-02_0001_20,C-02_0001,6


In [5]:
# add a ground truth column to the sc_profile dataframe based on if the track id is in the endpoint profile
sc_profile["Metadata_ground_truth_present"] = (
    sc_profile["Metadata_sc_unique_track_id"]
    .isin(sc_endpoint_profile["Metadata_sc_unique_track_id"])
    .astype(bool)
)
sc_profile

,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO,Metadata_sc_unique_track_id,Metadata_Well_FOV,Metadata_sc_unique_track_id_count,Metadata_ground_truth_present
0,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,101,...,1.126927,-0.143103,0.241127,-0.293259,-0.283715,1.434163,C-02_0001_17,C-02_0001,13,False
1,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,111,...,0.845704,0.083930,-1.990931,-0.030848,-1.033722,-0.942127,C-02_0001_18,C-02_0001,13,False
2,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,11,...,0.300940,-0.228780,1.782329,0.153739,-0.763335,0.725093,C-02_0001_5,C-02_0001,1,False
3,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,128,...,0.067196,-0.149771,1.405650,0.063245,2.162110,3.187469,C-02_0001_19,C-02_0001,13,False
4,1,C-02,180,Staurosporine,0.0,negative,1,0001,0.0,132,...,-1.851114,0.669517,-0.439855,1.576201,0.747753,0.895601,C-02_0001_20,C-02_0001,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182799,1,E-11,97,Staurosporine,156.25,test,13,0004,12.0,61,...,1.538514,-1.256247,-1.167025,2.543944,0.492066,-0.437865,E-11_0004_80,E-11_0004,11,False
182800,1,E-11,97,Staurosporine,156.25,test,13,0004,12.0,6,...,-0.289426,0.895888,-2.646345,1.653147,-0.763507,-1.567225,E-11_0004_11,E-11_0004,11,False
182801,1,E-11,97,Staurosporine,156.25,test,13,0004,12.0,62,...,2.786090,-0.056308,-0.942568,-0.867108,-0.909181,-0.240636,E-11_0004_147,E-11_0004,3,False
182802,1,E-11,97,Staurosporine,156.25,test,13,0004,12.0,64,...,2.720270,-0.602779,1.213263,-1.253546,-0.766996,2.020763,E-11_0004_86,E-11_0004,9,False


At this point there are two subsets to the dataset and will be split into the following datasplits:
- Single-cells that have a single cell ground truth
     - Holdout wells: 1/3 of wells
    - Train: 80% of the all cells with a single cell ground truth from the non-holdout wells
    - Validation:  10% of the all cells with a single cell ground truth from the non-holdout wells
    - Test: 10% of the all cells with a single cell ground truth from the non-holdout wells
- Single-cells that do not have a single cell ground truth
    - Holdout wells: 1/3 of wells
    - Test: 100% of the all cells with a single cell ground truth from the non-holdout wells


### hold out wells regardless of ground truth

In [6]:
index_data_split_and_ground_truth_dict = {
    "index": [],
    "data_split": [],
    "ground_truth": [],
}

In [7]:
# map the data_split to the sc_profile dataframe via the well
sc_profile["Metadata_data_split"] = sc_profile["Metadata_Well"].map(
    train_test_wells.set_index("Metadata_Well")["data_split"]
)
sc_profile.loc[sc_profile["Metadata_data_split"] == "test", "Metadata_data_split"] = (
    "well_holdout"
)
holdout_df = sc_profile.loc[sc_profile["Metadata_data_split"] == "well_holdout"]
index_data_split_and_ground_truth_dict["index"].append(holdout_df.index.tolist())
index_data_split_and_ground_truth_dict["data_split"].append(
    holdout_df["Metadata_data_split"].tolist()
)
index_data_split_and_ground_truth_dict["ground_truth"].append(
    holdout_df["Metadata_ground_truth_present"].tolist()
)
# get the non holdout wells
non_holdout_wells = sc_profile.loc[sc_profile["Metadata_data_split"] != "well_holdout"]
print(f"sc_profile shape after mapping data_split: {non_holdout_wells.shape}")
print(f"holdout_df shape: {holdout_df.shape}")

sc_profile shape after mapping data_split: (123348, 2378)
holdout_df shape: (59456, 2378)


### Cells that have a single cell ground truth

In [8]:
cell_wout_ground_truth_df = non_holdout_wells.loc[
    non_holdout_wells["Metadata_ground_truth_present"] == False
].copy()
cell_w_ground_truth_df = non_holdout_wells.loc[
    non_holdout_wells["Metadata_ground_truth_present"] == True
].copy()

print(f"cell_w_ground_truth_df shape: {cell_w_ground_truth_df.shape}")
print(f"cell_wout_ground_truth_df shape: {cell_wout_ground_truth_df.shape}")

cell_w_ground_truth_df shape: (6432, 2378)
cell_wout_ground_truth_df shape: (116916, 2378)


##

In [9]:
# Group by Metadata_sc_unique_track_id
grouped = cell_w_ground_truth_df.groupby("Metadata_sc_unique_track_id")

# Create a representative DataFrame for splitting
grouped_df = grouped.first().reset_index()

# Perform stratified split on the grouped data
train_groups, test_groups = train_test_split(
    grouped_df,
    test_size=0.2,
    stratify=grouped_df["Metadata_Well"],
    random_state=0,
)
test_groups, val_groups = train_test_split(
    test_groups,
    test_size=0.5,  # 50% of 20% is 10%
    stratify=test_groups["Metadata_Well"],
    random_state=0,
)

# Get the indices for each split
train_indices = cell_w_ground_truth_df[
    cell_w_ground_truth_df["Metadata_sc_unique_track_id"].isin(
        train_groups["Metadata_sc_unique_track_id"]
    )
].index
val_indices = cell_w_ground_truth_df[
    cell_w_ground_truth_df["Metadata_sc_unique_track_id"].isin(
        val_groups["Metadata_sc_unique_track_id"]
    )
].index
test_indices = cell_w_ground_truth_df[
    cell_w_ground_truth_df["Metadata_sc_unique_track_id"].isin(
        test_groups["Metadata_sc_unique_track_id"]
    )
].index

# Create the train, validation, and test DataFrames
train_sc_w_ground_truth_df = cell_w_ground_truth_df.loc[train_indices].copy()
val_sc_w_ground_truth_df = cell_w_ground_truth_df.loc[val_indices].copy()
test_sc_w_ground_truth_df = cell_w_ground_truth_df.loc[test_indices].copy()

# Assign metadata for data splits
train_sc_w_ground_truth_df["Metadata_data_split"] = "train"
train_sc_w_ground_truth_df["Metadata_ground_truth_present"] = True
val_sc_w_ground_truth_df["Metadata_data_split"] = "val"
val_sc_w_ground_truth_df["Metadata_ground_truth_present"] = True
test_sc_w_ground_truth_df["Metadata_data_split"] = "test"
test_sc_w_ground_truth_df["Metadata_ground_truth_present"] = True

# Print the shapes of the splits
print(f"train_sc_w_ground_truth_df shape: {train_sc_w_ground_truth_df.shape[0]}")
print(f"val_sc_w_ground_truth_df shape: {val_sc_w_ground_truth_df.shape[0]}")
print(f"test_sc_w_ground_truth_df shape: {test_sc_w_ground_truth_df.shape[0]}")

# Assertions to verify the splits
assert (
    train_sc_w_ground_truth_df.shape[0]
    + val_sc_w_ground_truth_df.shape[0]
    + test_sc_w_ground_truth_df.shape[0]
    == cell_w_ground_truth_df.shape[0]
)
assert (
    np.round(train_sc_w_ground_truth_df.shape[0] / cell_w_ground_truth_df.shape[0], 2)
    == 0.8
)
assert (
    np.round(val_sc_w_ground_truth_df.shape[0] / cell_w_ground_truth_df.shape[0], 2)
    == 0.1
)
assert (
    np.round(test_sc_w_ground_truth_df.shape[0] / cell_w_ground_truth_df.shape[0], 2)
    == 0.1
)

# Add to records
index_data_split_and_ground_truth_dict["index"].append(
    train_sc_w_ground_truth_df.index.tolist()
)
index_data_split_and_ground_truth_dict["data_split"].append(
    train_sc_w_ground_truth_df["Metadata_data_split"].tolist()
)
index_data_split_and_ground_truth_dict["ground_truth"].append(
    train_sc_w_ground_truth_df["Metadata_ground_truth_present"].tolist()
)
index_data_split_and_ground_truth_dict["index"].append(
    val_sc_w_ground_truth_df.index.tolist()
)
index_data_split_and_ground_truth_dict["data_split"].append(
    val_sc_w_ground_truth_df["Metadata_data_split"].tolist()
)
index_data_split_and_ground_truth_dict["ground_truth"].append(
    val_sc_w_ground_truth_df["Metadata_ground_truth_present"].tolist()
)
index_data_split_and_ground_truth_dict["index"].append(
    test_sc_w_ground_truth_df.index.tolist()
)
index_data_split_and_ground_truth_dict["data_split"].append(
    test_sc_w_ground_truth_df["Metadata_data_split"].tolist()
)
index_data_split_and_ground_truth_dict["ground_truth"].append(
    test_sc_w_ground_truth_df["Metadata_ground_truth_present"].tolist()
)

train_sc_w_ground_truth_df shape: 5146
val_sc_w_ground_truth_df shape: 641
test_sc_w_ground_truth_df shape: 645


#### Non tracked cells

In [10]:
cell_wout_ground_truth_df["Metadata_data_split"] = "test"
# add to records
index_data_split_and_ground_truth_dict["index"].append(
    cell_wout_ground_truth_df.index.tolist()
)
index_data_split_and_ground_truth_dict["data_split"].append(
    cell_wout_ground_truth_df["Metadata_data_split"].tolist()
)
index_data_split_and_ground_truth_dict["ground_truth"].append(
    cell_wout_ground_truth_df["Metadata_ground_truth_present"].tolist()
)
print(f"test_sc_wo_ground_truth_df shape: {cell_wout_ground_truth_df.shape[0]}")

test_sc_wo_ground_truth_df shape: 116916


### Fetch the indices from each ground truth and data split and add the status back to sc_profile

In [11]:
# flatten each list in the dictionar

for key in index_data_split_and_ground_truth_dict.keys():
    index_data_split_and_ground_truth_dict[key] = list(
        itertools.chain.from_iterable(index_data_split_and_ground_truth_dict[key])
    )
data_split_data_df = pd.DataFrame.from_dict(
    index_data_split_and_ground_truth_dict,
    orient="columns",
)
assert data_split_data_df.shape[0] == sc_profile.shape[0]

In [12]:
# sort the dataframe by index
data_split_data_df.sort_values(
    by=["index"],
    inplace=True,
)
# make the index the index column in data_split_data_df
data_split_data_df.set_index("index", inplace=True)
data_split_data_df.reset_index(drop=False, inplace=True)
data_split_data_df.head()

,index,data_split,ground_truth
0,0,well_holdout,False
1,1,well_holdout,False
2,2,well_holdout,False
3,3,well_holdout,False
4,4,well_holdout,False


In [13]:
# add the data_split and ground truth columns to the sc_profile dataframe by index
sc_profile_with_data_splits_df = pd.concat(
    [sc_profile, data_split_data_df],
    axis=1,
)
sc_profile_with_data_splits_df.drop(
    columns=["Metadata_data_split", "Metadata_ground_truth_present"],
    inplace=True,
)
sc_profile_with_data_splits_df.rename(
    columns={
        "data_split": "Metadata_data_split",
        "ground_truth": "Metadata_ground_truth_present",
    },
    inplace=True,
)

In [14]:
# final breakdown of the data
train_gt = sc_profile_with_data_splits_df[
    sc_profile_with_data_splits_df["Metadata_data_split"] == "train"
].copy()
train_gt = train_gt[train_gt["Metadata_ground_truth_present"] == True].copy()
val_gt = sc_profile_with_data_splits_df[
    sc_profile_with_data_splits_df["Metadata_data_split"] == "val"
].copy()
val_gt = val_gt[val_gt["Metadata_ground_truth_present"] == True].copy()
test_gt = sc_profile_with_data_splits_df[
    sc_profile_with_data_splits_df["Metadata_data_split"] == "test"
].copy()
test_gt = test_gt[test_gt["Metadata_ground_truth_present"] == True].copy()
test_wo_gt = sc_profile_with_data_splits_df[
    sc_profile_with_data_splits_df["Metadata_data_split"] == "test"
].copy()
test_wo_gt = test_wo_gt[test_wo_gt["Metadata_ground_truth_present"] == False].copy()
holdout_w_gt = sc_profile_with_data_splits_df[
    sc_profile_with_data_splits_df["Metadata_data_split"] == "well_holdout"
].copy()
holdout_w_gt = holdout_w_gt[
    holdout_w_gt["Metadata_ground_truth_present"] == True
].copy()
holdout_wo_gt = sc_profile_with_data_splits_df[
    sc_profile_with_data_splits_df["Metadata_data_split"] == "well_holdout"
].copy()
holdout_wo_gt = holdout_wo_gt[
    holdout_wo_gt["Metadata_ground_truth_present"] == False
].copy()
# assertion time :)
assert sc_profile_with_data_splits_df.shape[0] == sc_profile.shape[0]
assert (
    sc_profile_with_data_splits_df.shape[0]
    == train_gt.shape[0]
    + val_gt.shape[0]
    + test_gt.shape[0]
    + test_wo_gt.shape[0]
    + holdout_w_gt.shape[0]
    + holdout_wo_gt.shape[0]
)

In [15]:
print(f"train_gt shape: {train_gt.shape[0]}")
print(f"val_gt shape: {val_gt.shape[0]}")
print(f"test_gt shape: {test_gt.shape[0]}")
print(f"test_wo_gt shape: {test_wo_gt.shape[0]}")
print(f"holdout_w_gt shape: {holdout_w_gt.shape[0]}")
print(f"holdout_wo_gt shape: {holdout_wo_gt.shape[0]}")

train_gt shape: 5146
val_gt shape: 641
test_gt shape: 645
test_wo_gt shape: 116916
holdout_w_gt shape: 3220
holdout_wo_gt shape: 56236


In [16]:
data_split_data_df.head()

,index,data_split,ground_truth
0,0,well_holdout,False
1,1,well_holdout,False
2,2,well_holdout,False
3,3,well_holdout,False
4,4,well_holdout,False


## Now split the data into all splits including X, Y, and metadata
Each of the splits will saved to be loaded for hyperparameter optimization, training, and testing.

In [17]:
# read in the data
sc_file_path = pathlib.Path("../results/cleaned_sc_profile.parquet").resolve(
    strict=True
)
sc_endpoint_file_path = pathlib.Path(
    "../results/cleaned_endpoint_sc_profile.parquet"
).resolve(strict=True)


sc_profile = pd.read_parquet(sc_file_path)
sc_endpoint_profile = pd.read_parquet(sc_endpoint_file_path)
print(f"sc_profile shape: {sc_profile.shape}")
print(f"sc_endpoint_profile shape: {sc_endpoint_profile.shape}")

sc_profile shape: (182804, 2376)
sc_endpoint_profile shape: (11340, 368)


In [18]:
# merge the sc_profile and data_split_df
sc_profile = pd.concat(
    [
        sc_profile,
        data_split_data_df[["ground_truth", "data_split"]],
    ],
    axis=1,
)
sc_profile.rename(
    columns={
        "ground_truth": "Metadata_ground_truth",
        "data_split": "Metadata_data_split",
    },
    inplace=True,
)

In [19]:
# keep only the last timepoint
sc_profile["Metadata_Time"] = sc_profile["Metadata_Time"].astype("float64")
sc_profile = sc_profile[
    sc_profile["Metadata_Time"] == sc_profile["Metadata_Time"].max()
]
# drop Na values
sc_profile.dropna(inplace=True)
print(f"sc_profile shape after dropping NaN: {sc_profile.shape}")
sc_endpoint_profile.dropna(inplace=True)
print(f"sc_endpoint_profile shape after dropping NaN: {sc_endpoint_profile.shape}")

sc_profile shape after dropping NaN: (14237, 2378)
sc_endpoint_profile shape after dropping NaN: (11136, 368)


In [20]:
# hardcode the features that should exist in the y data
# this will be replaced in the future by an arg or config passed through
selected_y_features = ["Cells_Intensity_MeanIntensityEdge_AnnexinV"]
metadata_y_features = [x for x in sc_endpoint_profile.columns if "Metadata_" in x]
sc_endpoint_profile = sc_endpoint_profile[metadata_y_features + selected_y_features]

In [27]:
train_gt_X = sc_profile.loc[
    (sc_profile["Metadata_data_split"] == "train")
    & (sc_profile["Metadata_ground_truth"] == True)
]
val_gt_X = sc_profile.loc[
    (sc_profile["Metadata_data_split"] == "val")
    & (sc_profile["Metadata_ground_truth"] == True)
]
test_gt_X = sc_profile.loc[
    (sc_profile["Metadata_data_split"] == "test")
    & (sc_profile["Metadata_ground_truth"] == True)
]
test_wo_gt_X = sc_profile.loc[
    (sc_profile["Metadata_data_split"] == "test")
    & (sc_profile["Metadata_ground_truth"] == False)
]
holdout_w_gt_X = sc_profile.loc[
    (sc_profile["Metadata_data_split"] == "well_holdout")
    & (sc_profile["Metadata_ground_truth"] == True)
]
holdout_wo_gt_X = sc_profile.loc[
    (sc_profile["Metadata_data_split"] == "well_holdout")
    & (sc_profile["Metadata_ground_truth"] == False)
]
print(f"total cells: {sc_profile.shape[0]}")
print(f"train_gt_X shape: {train_gt_X.shape}")
print(f"val_gt_X shape: {val_gt_X.shape}")
print(f"test_gt_X shape: {test_gt_X.shape}")
print(f"test_wo_gt_X shape: {test_wo_gt_X.shape}")
print(f"holdout_w_gt_X shape: {holdout_w_gt_X.shape}")
print(f"holdout_wo_gt_X shape: {holdout_wo_gt_X.shape}")

total cells: 14237
train_gt_X shape: (372, 2378)
val_gt_X shape: (50, 2378)
test_gt_X shape: (49, 2378)
test_wo_gt_X shape: (9137, 2378)
holdout_w_gt_X shape: (244, 2378)
holdout_wo_gt_X shape: (4385, 2378)


In [29]:
total_gt_cells = (
    train_gt_X.shape[0]
    + val_gt_X.shape[0]
    + test_gt_X.shape[0]
    + holdout_w_gt_X.shape[0]
)
print(f"cells with ground truth: {total_gt_cells}")
print(
    f"Percentage of cells with ground truth: {total_gt_cells / sc_profile.shape[0] * 100:.2f}%"
)

cells with ground truth: 715
Percentage of cells with ground truth: 5.02%


In [22]:
# now let us get the the Metadata_sc_unique_track_id for each of the data splits with gt
train_df_x_Metadata_sc_unique_track_id = train_gt_X[
    "Metadata_sc_unique_track_id"
].unique()
val_df_x_Metadata_sc_unique_track_id = val_gt_X["Metadata_sc_unique_track_id"].unique()
test_df_x_Metadata_sc_unique_track_id = test_gt_X[
    "Metadata_sc_unique_track_id"
].unique()
holdout_w_df_x_Metadata_sc_unique_track_id = holdout_w_gt_X[
    "Metadata_sc_unique_track_id"
].unique()
print(
    f"train_df_x_Metadata_sc_unique_track_id shape: {train_df_x_Metadata_sc_unique_track_id.shape}"
)
print(
    f"val_df_x_Metadata_sc_unique_track_id shape: {val_df_x_Metadata_sc_unique_track_id.shape}"
)
print(
    f"test_df_x_Metadata_sc_unique_track_id shape: {test_df_x_Metadata_sc_unique_track_id.shape}"
)
print(
    f"holdout_w_df_x_Metadata_sc_unique_track_id shape: {holdout_w_df_x_Metadata_sc_unique_track_id.shape}"
)
# assertions :) make sure that the unique track ids are not overlapping
assert set(train_df_x_Metadata_sc_unique_track_id).isdisjoint(
    set(val_df_x_Metadata_sc_unique_track_id)
), "train and val track ids are overlapping"
assert set(train_df_x_Metadata_sc_unique_track_id).isdisjoint(
    set(test_df_x_Metadata_sc_unique_track_id)
), "train and test track ids are overlapping"
assert set(train_df_x_Metadata_sc_unique_track_id).isdisjoint(
    set(holdout_w_df_x_Metadata_sc_unique_track_id)
), "train and holdout track ids are overlapping"
assert set(val_df_x_Metadata_sc_unique_track_id).isdisjoint(
    set(test_df_x_Metadata_sc_unique_track_id)
), "val and test track ids are overlapping"
assert set(val_df_x_Metadata_sc_unique_track_id).isdisjoint(
    set(holdout_w_df_x_Metadata_sc_unique_track_id)
), "val and holdout track ids are overlapping"
assert set(test_df_x_Metadata_sc_unique_track_id).isdisjoint(
    set(holdout_w_df_x_Metadata_sc_unique_track_id)
), "test and holdout track ids are overlapping"

train_df_x_Metadata_sc_unique_track_id shape: (372,)
val_df_x_Metadata_sc_unique_track_id shape: (50,)
test_df_x_Metadata_sc_unique_track_id shape: (49,)
holdout_w_df_x_Metadata_sc_unique_track_id shape: (244,)


In [23]:
# find only the cell tracks that exist in the sc_profile
# train w gt
train_gt_y = sc_endpoint_profile.loc[
    sc_endpoint_profile["Metadata_sc_unique_track_id"].isin(
        train_df_x_Metadata_sc_unique_track_id
    )
].drop_duplicates("Metadata_sc_unique_track_id")
# val w gt
val_gt_y = sc_endpoint_profile.loc[
    sc_endpoint_profile["Metadata_sc_unique_track_id"].isin(
        val_df_x_Metadata_sc_unique_track_id
    )
].drop_duplicates("Metadata_sc_unique_track_id")
# test w gt
test_gt_y = sc_endpoint_profile.loc[
    sc_endpoint_profile["Metadata_sc_unique_track_id"].isin(
        test_df_x_Metadata_sc_unique_track_id
    )
].drop_duplicates("Metadata_sc_unique_track_id")
# holdout w gt
holdout_gt_y = sc_endpoint_profile.loc[
    sc_endpoint_profile["Metadata_sc_unique_track_id"].isin(
        holdout_w_df_x_Metadata_sc_unique_track_id
    )
].drop_duplicates("Metadata_sc_unique_track_id")


# find only cell tracks that exist in the endpoint profile
train_gt_X = train_gt_X.loc[
    train_gt_X["Metadata_sc_unique_track_id"].isin(
        train_gt_y["Metadata_sc_unique_track_id"]
    )
].drop_duplicates("Metadata_sc_unique_track_id")
val_gt_X = val_gt_X.loc[
    val_gt_X["Metadata_sc_unique_track_id"].isin(
        val_gt_y["Metadata_sc_unique_track_id"]
    )
].drop_duplicates("Metadata_sc_unique_track_id")
test_gt_X = test_gt_X.loc[
    test_gt_X["Metadata_sc_unique_track_id"].isin(
        test_gt_y["Metadata_sc_unique_track_id"]
    )
].drop_duplicates("Metadata_sc_unique_track_id")
holdout_w_gt_X = holdout_w_gt_X.loc[
    holdout_w_gt_X["Metadata_sc_unique_track_id"].isin(
        holdout_gt_y["Metadata_sc_unique_track_id"]
    )
].drop_duplicates("Metadata_sc_unique_track_id")

In [24]:
print(f"train_y_gt shape: {train_gt_y.shape}, train_gt_X shape: {train_gt_X.shape}")
print(f"val_y_gt shape: {val_gt_y.shape}, val_gt_X shape: {val_gt_X.shape}")
print(f"test_y_gt shape: {test_gt_y.shape}, test_gt_X shape: {test_gt_X.shape}")
print(
    f"holdout_y_gt shape: {holdout_gt_y.shape}, holdout_gt_X shape: {holdout_w_gt_X.shape}"
)
# assertions :) make sure that the number of unique samples are the same
# make sure that the X data does not overlap with other data splits
assert set(train_gt_X.index).isdisjoint(
    set(val_gt_X.index)
), "train and val data are overlapping"

assert set(train_gt_X.index).isdisjoint(
    set(test_gt_X.index)
), "train and test data are overlapping"
assert set(train_gt_X.index).isdisjoint(
    set(holdout_w_gt_X.index)
), "train and holdout data are overlapping"
assert set(val_gt_X.index).isdisjoint(
    set(test_gt_X.index)
), "val and test data are overlapping"
assert set(val_gt_X.index).isdisjoint(
    set(holdout_w_gt_X.index)
), "val and holdout data are overlapping"
assert set(test_gt_X.index).isdisjoint(
    set(holdout_w_gt_X.index)
), "test and holdout data are overlapping"


assert (
    train_gt_X.shape[0] == train_gt_y.shape[0]
), "train gt X and y shapes are not the same"
assert val_gt_X.shape[0] == val_gt_y.shape[0], "val gt X and y shapes are not the same"
assert (
    test_gt_X.shape[0] == test_gt_y.shape[0]
), "test gt X and y shapes are not the same"
assert (
    holdout_w_gt_X.shape[0] == holdout_gt_y.shape[0]
), "holdout gt X and y shapes are not the same"

train_y_gt shape: (367, 26), train_gt_X shape: (367, 2378)
val_y_gt shape: (49, 26), val_gt_X shape: (49, 2378)
test_y_gt shape: (49, 26), test_gt_X shape: (49, 2378)
holdout_y_gt shape: (237, 26), holdout_gt_X shape: (237, 2378)


In [25]:
# get metadata
metadata_X_cols = [x for x in train_gt_X.columns if "Metadata_" in x]
metadata_y_cols = [x for x in train_gt_y.columns if "Metadata_" in x]

# train
train_gt_X_metadata = train_gt_X[metadata_X_cols]
train_gt_X.drop(columns=metadata_X_cols, inplace=True)
train_gt_y_metadata = train_gt_y[metadata_y_cols]
train_gt_y.drop(columns=metadata_y_cols, inplace=True)
# val
val_gt_X_metadata = val_gt_X[metadata_X_cols]
val_gt_X.drop(columns=metadata_X_cols, inplace=True)
val_gt_y_metadata = val_gt_y[metadata_y_cols]
val_gt_y.drop(columns=metadata_y_cols, inplace=True)
# test
test_gt_X_metadata = test_gt_X[metadata_X_cols]
test_gt_X.drop(columns=metadata_X_cols, inplace=True)
test_gt_y_metadata = test_gt_y[metadata_y_cols]
test_gt_y.drop(columns=metadata_y_cols, inplace=True)
# holdout w gt
holdout_w_gt_X_metadata = holdout_w_gt_X[metadata_X_cols]
holdout_w_gt_X.drop(columns=metadata_X_cols, inplace=True)
holdout_w_gt_y_metadata = holdout_gt_y[metadata_y_cols]
holdout_gt_y.drop(columns=metadata_y_cols, inplace=True)

# note there is not wo gt for test and holdout
# get the indices
train_gt_X_metadata_index = train_gt_X_metadata.index
train_gt_y_metadata_index = train_gt_y_metadata.index
val_gt_X_metadata_index = val_gt_X_metadata.index
val_gt_y_metadata_index = val_gt_y_metadata.index
test_gt_X_metadata_index = test_gt_X_metadata.index
test_gt_y_metadata_index = test_gt_y_metadata.index
holdout_w_gt_X_metadata_index = holdout_w_gt_X_metadata.index
holdout_w_gt_y_metadata_index = holdout_w_gt_y_metadata.index

test_wo_gt_X_metadata_index = test_wo_gt_X.index
holdout_wo_gt_X_metadata_index = holdout_wo_gt_X.index
dict_of_index_lists = {
    "index_lists": [
        train_gt_X_metadata_index.tolist(),
        train_gt_y_metadata_index.tolist(),
        val_gt_X_metadata_index.tolist(),
        val_gt_y_metadata_index.tolist(),
        test_gt_X_metadata_index.tolist(),
        test_gt_y_metadata_index.tolist(),
        holdout_w_gt_X_metadata_index.tolist(),
        holdout_w_gt_y_metadata_index.tolist(),
        test_wo_gt_X_metadata_index.tolist(),
        holdout_wo_gt_X_metadata_index.tolist(),
    ],
    "data_split": [
        "train_gt",
        "train_gt",
        "val_gt",
        "val_gt",
        "test_gt",
        "test_gt",
        "holdout_gt",
        "holdout_gt",
        "test_wo_gt",
        "holdout_wo_gt",
    ],
    "data_x_or_y": [
        "X",
        "y",
        "X",
        "y",
        "X",
        "y",
        "X",
        "y",
        "X",
        "X",
    ],
}
dict_for_df = {
    "index": [],
    "data_split": [],
    "data_x_or_y": [],
}
for list_item in enumerate(dict_of_index_lists["index_lists"]):
    dict_for_df["index"].append(list_item[1])
    dict_for_df["data_split"].extend(
        [dict_of_index_lists["data_split"][list_item[0]]] * len(list_item[1])
    )
    dict_for_df["data_x_or_y"].extend(
        [dict_of_index_lists["data_x_or_y"][list_item[0]]] * len(list_item[1])
    )

# # flatten each list in the dictionary

dict_for_df["index"] = list(itertools.chain.from_iterable(dict_for_df["index"]))

# create a dataframe from the dictionary
index_data_split_and_x_y_df = pd.DataFrame.from_dict(
    dict_for_df,
    orient="columns",
)
index_data_split_and_x_y_df.head()

,index,data_split,data_x_or_y
0,7440,train_gt,X
1,7461,train_gt,X
2,7463,train_gt,X
3,7468,train_gt,X
4,7479,train_gt,X


In [26]:
# write the data splits to a parquet file
# this writes the indexes, ground truth, and data splits to a parquet file
# we do not write the sc_profile dataframe to a parquet file
data_split_file_path = pathlib.Path("../results/data_splits.parquet").resolve()
index_data_split_and_x_y_df.to_parquet(
    data_split_file_path,
    index=False,
)